In [10]:
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.initializers import Constant
from keras.layers import TextVectorization
import numpy as np
import pickle
import json

In [11]:
X = np.load('data/X.npy', allow_pickle=True)
y = np.load('data/y.npy')
embedding_matrix = np.load('data/embedding_matrix.npy')
num_tokens = np.load('data/num_tokens.npy').item()
embedding_dim = np.load('data/embedding_dim.npy').item()

In [12]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
vectorizer.adapt(X)

2022-02-01 14:47:35.226661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [13]:
y = y.reshape((-1,1))

In [14]:
print(X.shape)
print(y.shape)

(7613,)
(7613, 1)


In [15]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [19]:
np.save('data/x_train.npy', x_train)
np.save('data/y_train.npy', y_train)
np.save('data/x_test.npy', x_test)
np.save('data/y_test.npy', y_test)

In [17]:
sequences_input = Input(shape=(1), dtype='string')

vectorizer_sequence = vectorizer(sequences_input)

embedding_layer = Embedding(num_tokens,
                            embedding_dim,
                            embeddings_initializer=Constant(embedding_matrix),
                            trainable=False)
embedded_sequences = embedding_layer(vectorizer_sequence)

x = Dense(50, activation="relu")(embedded_sequences)
x = Dropout(0.2)(x)
x = Dense(20, activation="relu")(x)
x = Dropout(0.2)(x)
x = Dense(5, activation="relu")(x)
x = Dropout(0.2)(x)

preds = Dense(units=1,
              kernel_initializer='uniform',
              activation='sigmoid')(x)
model = Model(sequences_input, preds)

model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 200)              0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 200, 50)           888300    
                                                                 
 dense_4 (Dense)             (None, 200, 50)           2550      
                                                                 
 dropout_3 (Dropout)         (None, 200, 50)           0         
                                                                 
 dense_5 (Dense)             (None, 200, 20)           1020      
                                                           

In [18]:
n_samples = x_train.shape[0]

history = model.fit(x_train,
                    y_train,
                    validation_split=0.2,
                    batch_size=int(n_samples*0.2),
                    epochs=150)

model.save('model2.tf')

with open('history2.json', 'w') as f:
    json.dump(history.history, f)

Epoch 1/150


2022-02-01 14:47:35.863638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 3s 491ms/step - loss: 0.6930 - accuracy: 0.5600 - val_loss: 0.6928 - val_accuracy: 0.5751


2022-02-01 14:47:38.714544: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/150
5/5 [==============================] - 2s 442ms/step - loss: 0.6928 - accuracy: 0.5606 - val_loss: 0.6926 - val_accuracy: 0.5751
Epoch 3/150
5/5 [==============================] - 2s 440ms/step - loss: 0.6927 - accuracy: 0.5607 - val_loss: 0.6924 - val_accuracy: 0.5750
Epoch 4/150
5/5 [==============================] - 2s 440ms/step - loss: 0.6924 - accuracy: 0.5608 - val_loss: 0.6920 - val_accuracy: 0.5750
Epoch 5/150
5/5 [==============================] - 2s 443ms/step - loss: 0.6922 - accuracy: 0.5609 - val_loss: 0.6919 - val_accuracy: 0.5750
Epoch 6/150
5/5 [==============================] - 2s 441ms/step - loss: 0.6921 - accuracy: 0.5608 - val_loss: 0.6917 - val_accuracy: 0.5750
Epoch 7/150
5/5 [==============================] - 2s 433ms/step - loss: 0.6920 - accuracy: 0.5609 - val_loss: 0.6916 - val_accuracy: 0.5750
Epoch 8/150
5/5 [==============================] - 2s 441ms/step - loss: 0.6919 - accuracy: 0.5608 - val_loss: 0.6915 - val_accuracy: 0.5750
Epoch 9/150
5

2022-02-01 14:53:26.461269: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


INFO:tensorflow:Assets written to: model2.tf/assets
